# Welcome to Scheduler!

### Getting started
To begin, please choose a way you'd like to input your data. The first option allows you to load an excel spreadsheet with your own data. The second option loads some sample data for you to see the process and understand how the scheduling system works.


Run the command (shift+enter) below to ensure we are in the correct directory:

In [2]:
cd ../scheduler

C:\Users\dolly\Scheduler\scheduler


Run the command below to ensure we have the necessary packages needed to run our scheduler (pandas and xlrd):

In [3]:
!pip3 install pandas
!pip3 install xlrd

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


### Loading an Excel Spreadsheet

This is the first option, where you can load an excel sheet that contains your data. In this example, we will be loading from the excel file 'my_data.xl'. You can download, edit and update the excel sheet to reflect your data and reupload here. When this is done, run the cell below to properly load your data into the scheduling system!

In [4]:

import pandas as pd
from IPython.display import display 

df = pd.read_excel('my_data.xlsx')

prof_df = df.iloc[:,0:4]
prof_df['prof_courses'] = prof_df['prof_courses'].str.replace(', ',',')
rooms_df = df.iloc[:,4:6]
rooms_df.dropna(inplace=True)
rooms_df['room_capacity'] = rooms_df['room_capacity'].astype(u'int8')
rooms_df['rooms'] = rooms_df['rooms'].astype('str')
courses_df = df.iloc[:,6:]
courses_df.dropna(inplace=True)

professors = []
prof_info = {}
for index, row in prof_df.iterrows():
        data = {}
        prof = row['professor']
        professors.append(prof)
        courses = row['prof_courses'].split(',')
        data['courses'] = courses
        data['start_time'] = row['prof_start_time']
        data['end_time'] = row['prof_end_time']
        prof_info[prof] = data
        
rooms = []
room_capacities = {}
for _, row in rooms_df.iterrows():
    room = row['rooms']
    room_capacities[room] = row['room_capacity']
    rooms.append(room)

courses = []
course_no_students = {}
course_mins = {}
course_days_weekly = {}
for _, row in courses_df.iterrows():
    course = row['course']
    courses.append(course)
    course_no_students[course] = int(row['course_no_students'])
    course_mins[course] = int(row['course_mins'])
    course_days_weekly[course] = int(row['course_days_weekly'])
print("Data loaded successfully!")

Data loaded successfully!


Before continuing, you can run the commands below to ensure the professors, classrooms and courses you inputted are correct:

In [5]:
print('\n'.join(professors))

John Smith
Lisa Jones
Tim Simpson
Simon Valinski
Lisa Mullen
Jamir Abdullah'
Mike Williams
Rachel Smith
Gregg Woods
Chu Yen
Peter Parker
Elizabeth Walker
Brian K. Dickson


In [6]:
print('\n'.join(rooms))

655
666
745a
745b
433
201
115a
115b


In [7]:
print('\n'.join(courses))

physics
astrophysics
calculus 1
calculus 4
biology 1
biology 2
molecular biology
biochemistry
chemistry


### Loading Sample Data
This is the second option, where sample data is loaded instead of an Excel spreadsheet being used to retrieve data. 
Running the command below will generate the sample data. This will take data from a python file in which you can edit courses, teachers and classrooms, called 'datamaker.py' and export that generated data into a text file called 'sample_data.txt'.

In [8]:
import json
import datamaker

datamaker.make_data()

Now, we can run the command below to load the sample data to our program:

In [9]:
input_file = "sample_data.txt"
with open(input_file,'r') as f:
    data = json.load(f)
    
professors = data['professors']
prof_info = data['prof_info']
rooms = data['rooms']
room_capacities = data['room_capacities']
courses = data['courses']
course_no_students = data['course_no_students']
course_mins = data['course_mins']
course_days_weekly = data['course_days_weekly']

Before continuing, you can run the commands below to ensure the professors, classrooms and courses are correct:

In [10]:
print('\n'.join(professors))

John Smith
Lisa Jones
Mike Williams
Tim Simpson
Rachel Smith
Gregg Woods
Simon Valinski
Chu Yen
Peter Parker
Lisa Mullen
Elizabeth Walker
Brian K. Dickson
Jamir Abdullah


In [11]:
print('\n'.join(rooms))

655
666
745a
745b
433
201
115a
115b


In [12]:
print('\n'.join(courses))

physics
chemistry
biochemistry
biology 1
biology 2
molecular biology
calculus 1
calculus 4
astrophysics


The scheduler program will now take the courses shown above and find a professor, classroom and time that works.

### Scheduling the Teachers, Courses and Classrooms 

The following two cells will import necessary classes and format the data accordingly so that we can compute the class schedule:

In [16]:
from teachercourse_csp import assigner

In [17]:
def time_formatter(course, start_time):
    hs,ms = start_time
    end = hs*6 + ms//10 + course_mins[course]//10
    end_time = (end//6, (end - (end // 6) * 6) * 10)
    he,me = end_time
    output = '{:0>2}'.format(hs)+':'+'{:0>2}'.format(ms)+' to ' + '{:0>2}'.format(he)+':'+ '{:0>2}'.format(me) 
    return output

With the code block below, we can compute the class schedule.

In [20]:
user_data = (professors,prof_info,rooms,room_capacities,courses,course_no_students,course_mins,course_days_weekly)
full_schedule = assigner(user_data)
weekdays = ['mon','tues','wed','thur','fri']

columns = ['Day','Course','Professor', 'Room','Period']
df_out = pd.DataFrame(None, columns=columns)
for day in weekdays:
    schedule = full_schedule[day]
    print(day.upper())
    print('-----')
    for var,val in schedule.items():
        course,professor = var
        room,start_time = val
        df_inc = {'Day':day,'Course':[course],'Professor':[professor], 'Room':[room],'Period':[time_formatter(course,start_time)]}
        df_inc = pd.DataFrame.from_dict(df_inc)
        df_out = pd.concat([df_out,df_inc],ignore_index=True)
    print('')

MON
-----

TUES
-----

WED
-----

THUR
-----

FRI
-----



Run the command below to display the class schedule in order of day of week, course name, professor, the room and the time.

In [22]:
display(df_out)

Day             Course     Professor  Room          Period
0    mon            physics       Chu Yen  115b  16:00 to 17:00
1    mon       biochemistry  Peter Parker   201  13:30 to 15:00
2    mon          biology 1  Peter Parker  115b  13:30 to 15:00
3    mon          biology 2  Rachel Smith   433  11:30 to 12:30
4   tues          biology 2  Rachel Smith  115b  17:30 to 18:30
5    wed            physics       Chu Yen  115a  11:00 to 12:00
6    wed       biochemistry  Peter Parker   433  12:30 to 14:00
7    wed          biology 2  Rachel Smith  115b  10:00 to 11:00
8    wed  molecular biology  Peter Parker  115a  10:30 to 11:30
9   thur          biology 1  Peter Parker  115b  13:00 to 14:30
10  thur         calculus 4       Chu Yen   666  14:30 to 15:30
11   fri            physics       Chu Yen  745b  16:00 to 17:00
12   fri          biology 2  Rachel Smith  745b  10:30 to 11:30
13   fri         calculus 4       Chu Yen   655  14:00 to 15:00

You're done! Congratulations and thank you for using the Class Scheduler notebook!!